In [ ]:
import time

import pandas as pd
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
base_url = "https://www.ncbi.nlm.nih.gov"

In [ ]:
driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.get(base_url + "/gds/?term=aging+cell")

btn = driver.find_elements(By.ID, "EntrezSystem2.PEntrez.Gds.Gds_ResultsPanel.Gds_DisplayBar.Display")
btn[1].click()
p200 = driver.find_element(By.ID, "ps200")
time.sleep(1)
p200.click()

In [ ]:
parts = driver.find_elements(By.CLASS_NAME, "rprt")

result = []

In [ ]:
for p in parts:

    a = p.find_element(By.CSS_SELECTOR, ".title > a")
    href = a.get_attribute("href")
    title = a.text
    organ = p.find_element(By.CSS_SELECTOR, ".supp dd").text

    driver.execute_script(f"window.open('{href}', '_blank')")

    driver.switch_to.window(driver.window_handles[-1])
    source = driver.page_source
    has_raw = source.find("Raw data") != -1

    i = source.find("sra?term=")
    sra = None
    if i != -1:
        sra = source[i+9: i+18]

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    if has_raw:
        tmp = {
            "geo_accession": href.split("=")[1],
            "title": title,
            "organ": organ,
            "sra": sra
        }
        result.append(tmp)

In [ ]:
df = pd.DataFrame(result)
df.to_csv("o.csv")

In [ ]:
session = HTMLSession()
def find(addr):
    r = session.get(f"https://ngdc.cncb.ac.cn/search/?dbId=&q={addr}")
    has= True
    l =  r.html.find("div.ui.bottom.attached.active.tab.segment > p > strong")
    if len(l) >0:
        index = l[0].text[0]
        if int(index) <=0:
            has =False

    return  has

d = pd.read_csv("o.csv")
d["ngdc"] = d["geo_accession"].map(find)

In [ ]:
d.to_csv("new.csv")